<a href="https://www.kaggle.com/code/llkh0a/ensemble-inference?scriptVersionId=244409355" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

the traning process is in: https://www.kaggle.com/code/llkh0a/ensemble-training/

In [77]:
%ls /kaggle/input/ensemble-training/

catboost_info/  ensemble_model.joblib  __output__.json
custom.css      __notebook__.ipynb     __results__.html


In [84]:
%ls /kaggle/input/ensemble-cmo/other/default/1/

catboost_info/  ensemble_model.joblib


In [85]:
import os
import polars as pl
import pandas as pd
import numpy as np
import joblib

# Set the model path (Kaggle input directory)
# model_path = '/kaggle/input/ensemble-training/ensemble_model.joblib'
model_path = '/kaggle/input/ensemble-cmo/other/default/1/ensemble_model.joblib'
# Load the ensemble model
model = joblib.load(model_path)

# Load the label encoder (assumes it was saved as label_encoder.joblib in the same directory)
# import os
# le_path = os.path.join(os.path.dirname(model_path), 'label_encoder.joblib')
# if os.path.exists(le_path):
#     label_encoder = joblib.load(le_path)
# else:
#     label_encoder = None

may use the path /kaggle/input/ensemble-training/ensemble_model.joblib when i update the training notebook for updated model

since i forgor to save the label_encoder.joblib in the training notebook so i have to do hard code label map xd

In [48]:
label_map = {0: 'Above ear - pull hair', 1: 'Cheek - pinch skin', 2: 'Drink from bottle/cup', 3: 'Eyebrow - pull hair', 4: 'Eyelash - pull hair', 5: 'Feel around in tray and pull out an object', 6: 'Forehead - pull hairline', 7: 'Forehead - scratch', 8: 'Glasses on/off', 9: 'Neck - pinch skin', 10: 'Neck - scratch', 11: 'Pinch knee/leg skin', 12: 'Pull air toward your face', 13: 'Scratch knee/leg skin', 14: 'Text on phone', 15: 'Wave hello', 16: 'Write name in air', 17: 'Write name on leg'}

In [49]:
%cd /kaggle/working

/kaggle/working


# Feature Extraction and Inference API
This cell defines the feature extraction function and the predict function, following the competition's API format.

In [50]:
def extract_features(sequence: pl.DataFrame, demographics: pl.DataFrame) -> pd.DataFrame:
    # Accept both polars and pandas DataFrames
    if hasattr(sequence, 'to_pandas'):
        df = sequence.to_pandas()
    else:
        df = sequence
    if hasattr(demographics, 'to_pandas'):
        demo = demographics.to_pandas()
    else:
        demo = demographics
    feats = {}
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    exclude_cols = ['row_id', 'sequence_id', 'sequence_counter', 'subject']
    numeric_cols = [c for c in numeric_cols if c not in exclude_cols]
    for col in numeric_cols:
        feats[col + '_mean'] = df[col].mean()
        feats[col + '_std'] = df[col].std()
        feats[col + '_min'] = df[col].min()
        feats[col + '_max'] = df[col].max()
    for col in demo.columns:
        if col != 'subject':
            feats[col] = demo[col].iloc[0]
    for k, v in feats.items():
        if pd.isnull(v):
            feats[k] = 0
    return pd.DataFrame([feats])

def predict(sequence: pl.DataFrame, demographics: pl.DataFrame) -> str:
    X = extract_features(sequence, demographics)
    pred = model.predict(X)[0]
    # If label_encoder is available, decode the prediction
    # if label_encoder is not None:
    #     pred = label_encoder.inverse_transform([pred])[0]
    # return pred
    return label_map.get(pred, 'Unknown action')

# Example usage
This cell demonstrates how to use the predict function with sample data.

In [51]:
%ls /kaggle/input/cmi-detect-behavior-with-sensor-data/

kaggle_evaluation/  test_demographics.csv  train_demographics.csv
test.csv            train.csv


In [52]:
test_demo_df = pd.read_csv('/kaggle/input/cmi-detect-behavior-with-sensor-data/test_demographics.csv')
test_seq_df = pd.read_csv('/kaggle/input/cmi-detect-behavior-with-sensor-data/test.csv')

In [53]:
test_demo_df.head()

,subject,adult_child,age,sex,handedness,height_cm,shoulder_to_wrist_cm,elbow_to_wrist_cm
0,SUBJ_016452,1,25,1,1,165.0,52,23.0
1,SUBJ_055840,0,13,0,1,177.0,52,27.0


In [54]:
# Example (uncomment and adapt paths to use):
result = predict(test_seq_df, test_demo_df)
print('Predicted gesture:', result)

Predicted gesture: Neck - scratch


# Competition Submission: Inference Server Integration
Set up the inference server for competition submission. The server will use the trained ensemble model and the hardcoded label map to respond to inference requests.

In [56]:
import kaggle_evaluation.cmi_inference_server
import polars as pl
import os

def extract_features(sequence: pl.DataFrame, demographics: pl.DataFrame) -> pd.DataFrame:
    # Accept both polars and pandas DataFrames
    if hasattr(sequence, 'to_pandas'):
        df = sequence.to_pandas()
    else:
        df = sequence
    if hasattr(demographics, 'to_pandas'):
        demo = demographics.to_pandas()
    else:
        demo = demographics
    feats = {}
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    exclude_cols = ['row_id', 'sequence_id', 'sequence_counter', 'subject']
    numeric_cols = [c for c in numeric_cols if c not in exclude_cols]
    for col in numeric_cols:
        feats[col + '_mean'] = df[col].mean()
        feats[col + '_std'] = df[col].std()
        feats[col + '_min'] = df[col].min()
        feats[col + '_max'] = df[col].max()
    for col in demo.columns:
        if col != 'subject':
            feats[col] = demo[col].iloc[0]
    for k, v in feats.items():
        if pd.isnull(v):
            feats[k] = 0
    return pd.DataFrame([feats])


ModuleNotFoundError: No module named 'kaggle_evaluation'

In [ ]:

def predict(sequence: pl.DataFrame, demographics: pl.DataFrame) -> str:
    X = extract_features(sequence, demographics)
    pred = model.predict(X)[0]
    return label_map.get(pred, 'Unknown action')

inference_server = kaggle_evaluation.cmi_inference_server.CMIInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        data_paths=(
            '/kaggle/input/cmi-detect-behavior-with-sensor-data/test.csv',
            '/kaggle/input/cmi-detect-behavior-with-sensor-data/test_demographics.csv',
        )
    )